In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
print(os.environ["LANGCHAIN_TRACING_V2"])

[API KEY]
sk-proj-2ALiWBzcJl4s9ri6EUJ6T3BlbkFJxbUDIanlzbIf6JMPE7o2
true


In [3]:
import torch

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.chat import PromptTemplate, ChatPromptTemplate

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from transformers import AutoModel, AutoTokenizer

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager


# EEVE 모델 사용
llm = ChatOllama(
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    model="EEVE:latest",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [4]:
import os
current_directory = os.getcwd()
print(current_directory)

/home/choi/Git/RAG_con_doc/langchain


In [5]:
# 1단계 : 문서 로드
loader = PyMuPDFLoader("./data/Road_bridge_design_watergate.pdf")
docs = loader.load()
print(f"문서의 페이지수:{len(docs)}")


문서의 페이지수:6


In [6]:
# 2단계 : 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 7


In [7]:
# 3단계 : 임베딩 (결정)
# ko-sroberta-multitask 임베딩 사용
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

embeddings = HuggingFaceEmbeddings(model_name = 'jhgan/ko-sroberta-multitask')

sentences = [
    '관측된 세굴, 제방침식 및 잔해물이나 결빙체에 의한 구조물의 손상을 포함하는 과거 홍수시 구조물의 성능과 하천의 거동에 관해 입수 가능한 역사적 정보',
    '과거에 홍수가 났을 때 다리와 같은 구조물이 어떻게 버텼는지, 그리고 강이 어떻게 움직였는지에 대한 정보는, 홍수로 인해 생긴 땅의 침식이나 구조물의 손상 같은 관찰된 내용을 포함한다.',
    '세굴 해석에 요구되는 재료 특성을 분석하기에 충분한 깊이의 하상재료채취',
    ''
    ]
query_result1 = embeddings.embed_query(sentences[0])
query_result2 = embeddings.embed_query(sentences[1])
query_result3 = embeddings.embed_query(sentences[2])



/home/choi/anaconda3/envs/RAG_langchain/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# 3-1단계 : 임베딩 파인튜닝 (결정)
# klue roberta base 임베딩 사용
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

embeddings = HuggingFaceEmbeddings(model_name = '/home/choi/Git/ConSRoBERTa/output/con_nli_jhgan-ko-sbert-multitask-2024-07-17_21-06-49')

sentences = [
    '관측된 세굴, 제방침식 및 잔해물이나 결빙체에 의한 구조물의 손상을 포함하는 과거 홍수시 구조물의 성능과 하천의 거동에 관해 입수 가능한 역사적 정보',
    '과거에 홍수가 났을 때 다리와 같은 구조물이 어떻게 버텼는지, 그리고 강이 어떻게 움직였는지에 대한 정보는, 홍수로 인해 생긴 땅의 침식이나 구조물의 손상 같은 관찰된 내용을 포함한다.',
    '세굴 해석에 요구되는 재료 특성을 분석하기에 충분한 깊이의 하상재료채취',
    ''
    ]
query_result1 = embeddings.embed_query(sentences[0])
query_result2 = embeddings.embed_query(sentences[1])
query_result3 = embeddings.embed_query(sentences[2])


In [12]:
# jhgan/ko-sroberta-multitask 임베딩 유사도 비교
print(len(query_result1))

query_result1 = embeddings.embed_query(sentences[0])
query_result2 = embeddings.embed_query(sentences[1])
query_result3 = embeddings.embed_query(sentences[2])

query_result1  = torch.tensor(query_result1)
query_result2  = torch.tensor(query_result2)
query_result3  = torch.tensor(query_result3)

open_score01 = cal_score(query_result1, query_result2)
open_score02 = cal_score(query_result1, query_result3)

print(open_score01, open_score02)


768
tensor([[80.9746]]) tensor([[15.3142]])


In [18]:
# 3-1단계 : 임베딩 파인튜닝 (결정)
# jhgan ko sroberta 임베딩 사용
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

embeddings = HuggingFaceEmbeddings(model_name = '/home/choi/Git/ConSRoBERTa/output/con_nli_jhgan-ko-sbert-multitask-2024-07-17_21-14-24')

sentences = [
    '관측된 세굴, 제방침식 및 잔해물이나 결빙체에 의한 구조물의 손상을 포함하는 과거 홍수시 구조물의 성능과 하천의 거동에 관해 입수 가능한 역사적 정보',
    '과거에 홍수가 났을 때 다리와 같은 구조물이 어떻게 버텼는지, 그리고 강이 어떻게 움직였는지에 대한 정보는, 홍수로 인해 생긴 땅의 침식이나 구조물의 손상 같은 관찰된 내용을 포함한다.',
    '세굴 해석에 요구되는 재료 특성을 분석하기에 충분한 깊이의 하상재료채취',
    ''
    ]
query_result1 = embeddings.embed_query(sentences[0])
query_result2 = embeddings.embed_query(sentences[1])
query_result3 = embeddings.embed_query(sentences[2])

In [19]:
# jhgan/ko-sroberta-multitask 임베딩 유사도 비교
print(len(query_result1))

query_result1 = embeddings.embed_query(sentences[0])
query_result2 = embeddings.embed_query(sentences[1])
query_result3 = embeddings.embed_query(sentences[2])

query_result1  = torch.tensor(query_result1)
query_result2  = torch.tensor(query_result2)
query_result3  = torch.tensor(query_result3)

open_score01 = cal_score(query_result1, query_result2)
open_score02 = cal_score(query_result1, query_result3)

print(open_score01, open_score02)


768
tensor([[78.1219]]) tensor([[31.5089]])


In [29]:
# 3단계-1 : 임베딩
# KoSimCSE-roberta 임베딩 사용
model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta-multitask') 
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta-multitask')

inputs = tokenizer(split_documents[0].page_content, padding=True, truncation=True, return_tensors='pt')
embeddings, _ = model(**inputs, return_dict=False)

print(len(embeddings[0][0]))

768


In [31]:
# KoSimCSE-roberta 임베딩 유사도 비교
# 간이 건설 Semantic Textual Similarity test
def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

sentences = [
    '관측된 세굴, 제방침식 및 잔해물이나 결빙체에 의한 구조물의 손상을 포함하는 과거 홍수시 구조물의 성능과 하천의 거동에 관해 입수 가능한 역사적 정보',
    '과거에 홍수가 났을 때 다리와 같은 구조물이 어떻게 버텼는지, 그리고 강이 어떻게 움직였는지에 대한 정보는, 홍수로 인해 생긴 땅의 침식이나 구조물의 손상 같은 관찰된 내용을 포함한다.',
    '세굴 해석에 요구되는 재료 특성을 분석하기에 충분한 깊이의 하상재료채취',
    ''
    ]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
embeddings, _ = model(**inputs, return_dict=False)

score01 = cal_score(embeddings[0][0], embeddings[1][0])
score02 = cal_score(embeddings[0][0], embeddings[2][0])
print(score01, score02)

tensor([[85.9120]], grad_fn=<MulBackward0>) tensor([[46.9270]], grad_fn=<MulBackward0>)


In [52]:
# 3단계-2 : 임베딩 OpenAI embedding 사용
def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

embeddings2 = OpenAIEmbeddings(model = 'text-embedding-ada-002')
sentences = [
    '관측된 세굴, 제방침식 및 잔해물이나 결빙체에 의한 구조물의 손상을 포함하는 과거 홍수시 구조물의 성능과 하천의 거동에 관해 입수 가능한 역사적 정보',
    '과거에 홍수가 났을 때 다리와 같은 구조물이 어떻게 버텼는지, 그리고 강이 어떻게 움직였는지에 대한 정보는, 홍수로 인해 생긴 땅의 침식이나 구조물의 손상 같은 관찰된 내용을 포함한다.',
    '세굴 해석에 요구되는 재료 특성을 분석하기에 충분한 깊이의 하상재료채취',
    ''
    ]
query_result1 = embeddings2.embed_query(sentences[0])
query_result2 = embeddings2.embed_query(sentences[1])
query_result3 = embeddings2.embed_query(sentences[2])



In [39]:
# OpenAI 임베딩 유사도 비교
print(len(query_result1))

query_result1 = embeddings2.embed_query(sentences[0])
query_result2 = embeddings2.embed_query(sentences[1])
query_result3 = embeddings2.embed_query(sentences[2])

query_result1  = torch.tensor(query_result1)
query_result2  = torch.tensor(query_result2)
query_result3  = torch.tensor(query_result3)

open_score01 = cal_score(query_result1, query_result2)
open_score02 = cal_score(query_result1, query_result3)

print(open_score01, open_score02)



1536
tensor([[90.9867]]) tensor([[83.3793]])


In [7]:
# 3단계-3 임베딩 GPT4ALL nomic-embed 사용
from langchain_community.embeddings import OllamaEmbeddings

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

embeddings3 = OllamaEmbeddings(model='nomic-embed-text:latest')
sentences = [
    '관측된 세굴, 제방침식 및 잔해물이나 결빙체에 의한 구조물의 손상을 포함하는 과거 홍수시 구조물의 성능과 하천의 거동에 관해 입수 가능한 역사적 정보',
    '과거에 홍수가 났을 때 다리와 같은 구조물이 어떻게 버텼는지, 그리고 강이 어떻게 움직였는지에 대한 정보는, 홍수로 인해 생긴 땅의 침식이나 구조물의 손상 같은 관찰된 내용을 포함한다.',
    '세굴 해석에 요구되는 재료 특성을 분석하기에 충분한 깊이의 하상재료채취',
    ''
    ]
query_result1 = embeddings3.embed_query(sentences[0])
query_result2 = embeddings3.embed_query(sentences[1])
query_result3 = embeddings3.embed_query(sentences[2])

In [9]:
# nomic-embed 임베딩 유사도 비교
print(len(query_result1))

# query_result1 = embeddings.embed_query(sentences[0])
# query_result2 = embeddings.embed_query(sentences[1])
# query_result3 = embeddings.embed_query(sentences[2])

query_result1  = torch.tensor(query_result1)
query_result2  = torch.tensor(query_result2)
query_result3  = torch.tensor(query_result3)

open_score01 = cal_score(query_result1, query_result2)
open_score02 = cal_score(query_result1, query_result3)

print(open_score01, open_score02)

768
tensor([[87.3962]]) tensor([[92.1106]])


In [10]:
# 단계 4: DB 생성 및 저장
# vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings3)

In [11]:
# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [13]:
# 검색기 테스트
retriever.invoke("암거 위치, 길이와 수로면적을 설계할시 고려해야할 사항은?")

[Document(page_content='설계시 고려해야 한다.\n  난류 수역에 위치한 교대의 안정성에 대해서는 철저한 검토를 수행해야 한다. 노출된 제방경사\n지는 적절한 세굴방지대책으로 보호한다.', metadata={'source': './data/Road_bridge_design_watergate.pdf', 'file_path': './data/Road_bridge_design_watergate.pdf', 'page': 3, 'total_pages': 6, 'format': 'PDF 1.6', 'title': '<3132C2F7BFE4BEE0BABB2DC3D6C1BEC0CEBCE232303132303230382E687770>', 'author': '<BEC6C6AE>', 'subject': '', 'keywords': '', 'creator': 'PScript5.dll Version 5.2', 'producer': 'Acrobat Distiller 7.0.5 (Windows)', 'creationDate': "D:20240627141630+09'00'", 'modDate': "D:20240627141712+09'00'", 'trapped': ''}),
 Document(page_content='제2장\n설계개요\n2-18 ●\n(2) 설계강우강도\n  교량바닥판 배수에 적용하는 설계강우강도는 발주자가 규정하지 않는 한 인접도로의 포장 배\n수설계에 적용하는 설계강우강도보다 작지 않아야 한다.\n(3) 배수시설의 형식, 규격 및 개수\n  바닥판배수시설의 개수는 수리조건을 만족시키는 범위에서 최소로 한다.\n  편경사가 변하는 곳에서는 배수흐름을 고려한 등고선을 작성하여 신속하게 교면수를 유도할 \n수 있도록 교면배수시설을 설치하여야 한다.\n  바닥판 배수시설의 집수구는 수리학적으로 효과적이고 청소를 위한 접근이 가능해야 한다.\n(4) 바닥판 배수시설로부터의 유출\n  바닥판 배수시설은 바닥판이나 노면의 지표수가 교량 상부구조부재와 하부구조로부터 

In [14]:
# 단계 6: 프롬프트 생성(Create Prompt)
prompt = ChatPromptTemplate.from_template(
    '''You are a helpful assistant, conversing with a user about the subjects contained in a set of documents.
Use the information from the DOCUMENTS section to provide accurate answers. If unsure or if the answer
isn't found in the DOCUMENTS section, simply state that you don't know the answer. Please answer me in korean. 
#Question:
{question}
#Context:
{context}
'''
)

In [15]:
# 단계 7: LLM 생성
llm = ChatOllama(model='EEVE:latest')
# llm = ChatOpenAI(model_name='gpt-4o')
# llm = ChatOllama(model='llama3_kor:latest')

In [16]:
# 단계 8: 체인 생성
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
# 질문 실행

question = "암거 위치, 길이와 수로면적을 설계할시 고려해야할 사항은?"

response = chain.invoke(question)
print(response)

다음은 PDF 파일 텍스트 내용입니다:

2.6 수문 및 수리
●2-13
아래의 지침들을 고려하세요:
교량의 설치 장소를 선정하는 단계에서, 교각이 적거나 없는 설계안을 검토하고 기본 설계 단계에서 자세히 논의합니다.
교각의 형상이 서로 일치하고 상부 구조와 균형을 이루도록 합니다.
구성요소의 형상과 구조형식의 급격한 변화를 피합니다. 다른 구조형식 사이의 연결이 불가피한 경우 외관상 자연스럽게 보이게 합니다.
바닥판 배수 홈통과 같은 상세 설계에 주의를 기울입니다.
기능성 또는 경제적 관점에서 하로트러스교와 같은 교량 유형이 불가피한 경우에는 개방적이고 혼잡하지 않은 외관을 제공하는 구조체를 선택합니다.
교량을 교통 표지판, 조명등 또는 현수막을 지지하는 구조물로 사용하는 것을 가능한 한 피합니다.
지점 이외에 설치된 횡방향 복부판 보강재는 측면에서 보이지 않도록 합니다.
2.6 수문 및 수리
2.6.1 일반사항
하천을 횡단하는 교량 부지에 대한 수리수문학적인 검토와 평가는 기본계획 개발의 일부로 이루어져야 합니다. 이러한 검토의 세부 사항들은 구조물의 중요성 및 연관 위험성과 동등하게 취급합니다.
시공자 또는 공사 중 교통 소통용 임시 구조물은 범람원 자연 자원에 미치는 영향을 최소화하고 인접 시설 소유주 및 대중교통 안전성을 확보하기 위해 설계됩니다.
2.6.5 암거 위치, 길이 및 수로 면적
2.6.3과 2.6.4의 규정을 포함하여 아래 조건들을 고려하세요:
- 어류와 야생동물의 통로
- 큰 출구부 유속 및 흐름 집중이 암거출구부, 하류수로 및 인접 시설물에 미치는 영향
- 암거입구부에 작용하는 부력 효과
- 교통 안전
- 하류통제와 폭우고조에 따른 하류 고수위의 영향
2.6.6 도로 배수
(1) 일반사항
통행 차량의 안전을 극대화하고 교량의 손상을 최소화하기 위해 교량 바닥판과 진입로는 노면수를 효과적으로 안전하게 배수할 수 있도록 설계되어야 합니다. 차도, 자전거 이용도로 및 보도를 포함한 바닥판의 횡단배수는 충분히 자연배수가 가능하도록 횡단경사 또

In [9]:
print(llm.invoke("1더하고 1이 뭐야?"))

"1 더하기 1은 무엇인가?"라는 질문에 대한 답변입니다.

1더하기1(1+1)의 결과는 2입니다. 간단히 말해서, 한 개체에 하나를 더하면 합계나 총수가 됩니다. 이 경우, 두 개의 단위('1'과 '1')를 더한 결과물이 2가 되는 것이죠. 이 개념은 기본적인 산수에서 중요한 역할을 하며 일상 생활의 다양한 측면에 적용됩니다. 예를 들어, 물건이나 사람 수 세기, 계량 문제 해결, 간단한 계산 등입니다.

이것을 기억하기 위해서, 다음과 같은 공식을 사용할 수 있습니다:

1+1 = 2

수학에서는 이를 '덧셈'이라고 하는데, 이는 숫자들을 결합하거나 합쳐서 새로운 총수를 만드는 것을 의미합니다. 이 개념은 더 복잡한 수학 과정의 기초로 사용되며, 다양한 유형의 산술 연산에 적용됩니다.content='"1 더하기 1은 무엇인가?"라는 질문에 대한 답변입니다.\n\n1더하기1(1+1)의 결과는 2입니다. 간단히 말해서, 한 개체에 하나를 더하면 합계나 총수가 됩니다. 이 경우, 두 개의 단위(\'1\'과 \'1\')를 더한 결과물이 2가 되는 것이죠. 이 개념은 기본적인 산수에서 중요한 역할을 하며 일상 생활의 다양한 측면에 적용됩니다. 예를 들어, 물건이나 사람 수 세기, 계량 문제 해결, 간단한 계산 등입니다.\n\n이것을 기억하기 위해서, 다음과 같은 공식을 사용할 수 있습니다:\n\n1+1 = 2\n\n수학에서는 이를 \'덧셈\'이라고 하는데, 이는 숫자들을 결합하거나 합쳐서 새로운 총수를 만드는 것을 의미합니다. 이 개념은 더 복잡한 수학 과정의 기초로 사용되며, 다양한 유형의 산술 연산에 적용됩니다.' response_metadata={'model': 'EEVE:latest', 'created_at': '2024-06-27T15:14:49.150763199Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 3378069120, 'load_duratio